# test de traitement de data dans un fichier TDMS de LabView

Objectif : faire du post-traitement sur un fichier de type TDMS (origine d'une carte d'acquisition DAQ sous Labview)

le fichier de data est une trame (acquisition analogique d'un signal de télecommande de type RC5) : remise en forme, re-échantillonnage et extraction des bits

- étape initiale installation de la bibliothèque avec la commande `pip install npTDMS'
- vérification de sa version avec la commande suivante

In [ ]:
%pip show npTDMS

- ouverture du fichier de data TDMS (modifier le chemin en fonction de votre machine)
- le fichier ne possède qu'une seule mesure de tension.. vous pouvez l'importer avec LabView, mais la visualisation est aussi réalisé en python
- les noms des "group" et "channel" je les ai trouvés dans le fichier index !! ceux de l'exemple ne sont pas bon

In [ ]:
from nptdms import TdmsFile

#file_path = r"C:\Users\courbon\Documents\jupyter\LV\test.tdms"
file_path = "./test.tdms"
tdms_file = TdmsFile.read(file_path)
group = tdms_file['Group Name']
channel = group['Voltage']
channel_data = channel[:]
channel_properties = channel.properties
print ("properties:",channel_properties,"\n")
print ("data",channel_data)

- quelques caractéristiques du signal (waveform) comme l'échantillonnage et la taille

In [ ]:
dt = channel_properties["wf_increment"]
nbData = len(channel_data)
print ("Te = ",dt,"s", " ; size ",nbData)

- affichage du signal avec matplotlib (d'abord en mode ligne), cf la documentation sur le site de ce projet à https://matplotlib.org/ ou plus simplement à https://www.w3schools.com/python/matplotlib_intro.asp 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(channel_data)
plt.show()

- réalisation d'une segmentation simple et affichage du résultat

In [ ]:
bin_data = []
for x in channel_data:
    if x < 4.7 :
        x = 0
    else:
        x = 1
    bin_data.append(x)
plt.plot(bin_data)
plt.show()

- ensuite, ré-echantillonage du tableau : pour chaque bit prendre une valeur au milieu de la 1/2 période, voir la formule de calcul sur la doc Joplin

In [ ]:
import math
Te = dt * 1000
table1 = []
table2 = []
for nb_bit in range(0,14):
    i1 = math.trunc( (0.445+nb_bit*1.778) / Te)
    table1.append(bin_data[i1])
    i2 = math.trunc( (1.3335+nb_bit*1.778) / Te)
    table2.append(bin_data[i2])
    print( 'indices = ',i1,i2)

for i in range(0,13):
    print('t1=',table1[i],'/t t2 = ',table2[i])

- construction des valeurs de l'adressage (sur 5 bits) et de l'instruction (sur les 6 bits suivants) et la transformation d'une liste ou array de binaire en décimal cf https://www.tutorialspoint.com/binary-list-to-integer-in-python

In [ ]:
deca = 3 
adresse = []
for i in range(0,5):
    adresse.append(table1[i+deca])
deca += 5
instruction = []
print( 'adresse = ',adresse)
result = 0
for digits in adresse:
    result = (result << 1) | digits
print( 'en decimal ',result)

for i in range(0,6):
        instruction.append(table1[i+deca])
print( 'instruction = ',instruction)
result = 0
for digits in instruction:
    result = (result << 1) | digits
print( 'en decimal ',result)